# Default Import

In [1]:
from ris import pysqldb
from IPython.display import Markdown, clear_output
from sqlalchemy import create_engine
import ris
import getpass
import datetime 
import pandas as pd
import numpy as np
import os
import requests

timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

Markdown('<strong>Notebook run on: {} | by {} | Using ris library version: {} | File Location: {}'.format(
    timestamp, getpass.getuser(), ris.__version__, os.getcwd()
))


<strong>Notebook run on: 2020-09-10 17:16 | by soge | Using ris library version: 1.5.6 | File Location: E:\RIS\Staff Folders\Samuel\Requests\AG\SIP Requests\SIP_bike_lanes_2020

# Db connections

In [2]:
try: 
    db.params['user']
except:
    db = pysqldb.DbConnect(type='PG', server='dotpgsql01', database='sip')


Additional database connection details required:
User name (sip):soge
Password (sip)········
Database connection (PG) to sip on dotpgsql01 - user: soge 
Connection established 2020-09-10 17:16:12


In [3]:
#sip = pysqldb.DbConnect(server='dotpgsql01', database='sip', type='PG', user = db.params['user'], password = db.params['password'])
#forms = pysqldb.DbConnect(server='dot55sql01', database='forms', type='MS', user = 'arcgis', password = 'arcgis')

In [4]:
db.connect()

Database connection (PG) to sip on dotpgsql01 - user: soge 
Connection established 2020-09-10 17:16:13


In [5]:
#complicated way

data = db.dfquery("""   -- names, treatments, and mileage of all 2020 installed bike lanes

                        WITH data AS(
                        SELECT tree.descendant AS qid, 
                        array_to_string(array_agg(tree.concat), ' -> '::text) AS full_treatment,
                        array_to_string(array_agg(tree.ancestor), ' -> '::text) AS full_treatment_int
                        FROM (  SELECT p.descendant, p.ancestor, q.question, p.depth, 
                               pg_catalog.concat(p.ancestor, ': ', q.question, ' (Depth:', p.depth, ')') AS concat
                            FROM sip_questions_path p
                            JOIN sip_questions q ON p.ancestor = q.qid
                            ORDER BY p.descendant, p.depth DESC) tree
                        GROUP BY tree.descendant
                        )

                        SELECT CASE WHEN data.full_treatment_int like '5 -> 258 -> 259%'
                                                              OR full_treatment_int like '5 -> 258 -> 260%'
                                                              OR full_treatment_int like '5 -> 258 -> 261%' THEN '1' 
                                    WHEN data.full_treatment_int like '5 -> 258 -> 262%' THEN '2'
                                    WHEN data.full_treatment_int like '5 -> 258 -> 263%'
                                                              OR full_treatment_int like '5 -> 258 -> 463%' THEN '3' END AS Class,
                               sum(coalesce(st_length(cl.geom)/5280,0)) mileage
                        FROM public.sip_treatments st
                        LEFT JOIN data
                        ON data.qid = st.breadcrumbid
                        LEFT JOIN public.sip_projects sp
                        ON st.pid_fk = sp.pid
                        LEFT JOIN public.sip_treatments_geo stg
                        ON st.tid = stg.tid_fk
                        LEFT JOIN public.clion cl
                        ON stg.segmentid::int = cl.segmentid::int
                        JOIN sip_lookup sl
                        ON sp.status::int = sl.lookupid
                        WHERE full_treatment_int like '5 -> 258%'
                        AND sip_year = 2020
                        AND sp.status = '15'
                        GROUP BY CASE WHEN data.full_treatment_int like '5 -> 258 -> 259%'
                                                              OR full_treatment_int like '5 -> 258 -> 260%'
                                                              OR full_treatment_int like '5 -> 258 -> 261%' THEN '1'
                                    WHEN data.full_treatment_int like '5 -> 258 -> 262%' THEN '2'
                                    WHEN data.full_treatment_int like '5 -> 258 -> 263%'
                                                              OR full_treatment_int like '5 -> 258 -> 463%' THEN '3' END
                                 
""")

In [6]:
db.connect()

Database connection (PG) to sip on dotpgsql01 - user: soge 
Connection established 2020-09-10 17:16:14


In [7]:
#Simple way

In [8]:
sip_data = db.dfquery("""   -- names, treatments, and mileage of all 2020 installed bike lanes

                        WITH data AS(
                        SELECT tree.descendant AS qid, 
                        array_to_string(array_agg(tree.concat), ' -> '::text) AS full_treatment,
                        array_to_string(array_agg(tree.ancestor), ' -> '::text) AS full_treatment_int
                        FROM (  SELECT p.descendant, p.ancestor, q.question, p.depth, 
                               pg_catalog.concat(p.ancestor, ': ', q.question, ' (Depth:', p.depth, ')') AS concat
                            FROM sip_questions_path p
                            JOIN sip_questions q ON p.ancestor = q.qid
                            ORDER BY p.descendant, p.depth DESC) tree
                        GROUP BY tree.descendant
                        )

                        SELECT CASE WHEN data.full_treatment like '%Bikes% -> %New Facility% -> %Protected%' THEN '1'
                                    WHEN data.full_treatment like '%Bikes% -> %New Facility% -> %Standard%'
                                                              OR data.full_treatment like '%Bikes% -> %New Facility% -> %Buffered%'
                                                              OR data.full_treatment like '%Bikes% -> %New Facility% -> %Curbside%' THEN '2' 
                                    WHEN data.full_treatment like '%Bikes% -> %New Facility% -> %Sharrows%'
                                                              OR data.full_treatment_int like '5: Bikes % -> %New Facility% -> %Parking%' THEN '3' END AS Class,
                               sum(coalesce(st_length(cl.geom)/5280,0)) mileage
                        FROM public.sip_treatments st
                        LEFT JOIN data
                        ON data.qid = st.breadcrumbid
                        LEFT JOIN public.sip_projects sp
                        ON st.pid_fk = sp.pid
                        LEFT JOIN public.sip_treatments_geo stg
                        ON st.tid = stg.tid_fk
                        LEFT JOIN public.clion cl
                        ON stg.segmentid::int = cl.segmentid::int
                        JOIN sip_lookup sl
                        ON sp.status::int = sl.lookupid
                        WHERE full_treatment_int like '5 -> 258%'
                        AND sip_year = 2020
                        AND sp.status = '15'
                        GROUP BY CASE WHEN data.full_treatment like '%Bikes% -> %New Facility% -> %Protected%' THEN '1'
                                      WHEN data.full_treatment like '%Bikes% -> %New Facility% -> %Standard%'
                                                              OR data.full_treatment like '%Bikes% -> %New Facility% -> %Buffered%'
                                                              OR data.full_treatment like '%Bikes% -> %New Facility% -> %Curbside%' THEN '2' 
                                      WHEN data.full_treatment like '%Bikes% -> %New Facility% -> %Sharrows%'
                                                              OR data.full_treatment_int like '5: Bikes % -> %New Facility% -> %Parking%' THEN '3' END                                
""")

In [9]:
sip_data

,class,mileage
0,1,0.935072
1,2,13.452089


In [11]:
#data.to_csv('sip_installed_bike_lanes_2020_{ts}.csv'.format(ts=ts), index = False)